In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Flatten, Input
from tensorflow.keras.optimizers import Adam, schedules
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [4]:
train_data_gen = ImageDataGenerator(rescale=1./255)
validation_data_gen = ImageDataGenerator(rescale=1./255)

In [5]:
train_generator = train_data_gen.flow_from_directory('./data/train', target_size=(48,48), batch_size=64, color_mode="grayscale", class_mode='categorical')

Found 28709 images belonging to 7 classes.


In [6]:
validation_generator = validation_data_gen.flow_from_directory('./data/test', target_size=(48,48), batch_size=64, color_mode="grayscale", class_mode='categorical')

Found 7178 images belonging to 7 classes.


In [12]:
emotion_model = Sequential()

emotion_model.add(Input(shape=(48, 48, 1)))

emotion_model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
emotion_model.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2, 2)))
emotion_model.add(Dropout(0.25))

In [13]:
emotion_model.add(Flatten())
emotion_model.add(Dense(1024, activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7, activation='softmax'))

In [23]:
initial_learning_rate = 0.001
decay_steps = 10000
decay_rate = 0.96

lr_schedule = schedules.ExponentialDecay(initial_learning_rate, decay_steps=decay_steps, decay_rate=decay_rate, staircase=True)

emotion_model.compile(loss='categorical_crossentropy', optimizer=Adam(learning_rate=lr_schedule), metrics=['accuracy'])

In [25]:
emotion_model_info = emotion_model.fit(train_generator, steps_per_epoch=28709//64, epochs=50, validation_data=validation_generator, validation_steps=7178//64)

C:\ProgramData\anaconda3\Lib\site-packages\keras\src\trainers\data_adapters\py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 122s 269ms/step - accuracy: 0.2554 - loss: 1.7954 - val_accuracy: 0.4062 - val_loss: 1.5565
Epoch 2/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 240us/step - accuracy: 0.3750 - loss: 1.6310 - val_accuracy: 0.3000 - val_loss: 1.5798
Epoch 3/50


C:\ProgramData\anaconda3\Lib\contextlib.py:158: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(value)


448/448 ━━━━━━━━━━━━━━━━━━━━ 85s 190ms/step - accuracy: 0.4151 - loss: 1.5049 - val_accuracy: 0.4972 - val_loss: 1.3063
Epoch 4/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 169us/step - accuracy: 0.5000 - loss: 1.3931 - val_accuracy: 0.5000 - val_loss: 1.5597
Epoch 5/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 95s 213ms/step - accuracy: 0.4912 - loss: 1.3372 - val_accuracy: 0.5146 - val_loss: 1.2535
Epoch 6/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 167us/step - accuracy: 0.4688 - loss: 1.2886 - val_accuracy: 0.5000 - val_loss: 1.3671
Epoch 7/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 96s 213ms/step - accuracy: 0.5290 - loss: 1.2437 - val_accuracy: 0.5543 - val_loss: 1.1830
Epoch 8/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 233us/step - accuracy: 0.5312 - loss: 1.2523 - val_accuracy: 0.8000 - val_loss: 1.2166
Epoch 9/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 108s 241ms/step - accuracy: 0.5463 - loss: 1.1821 - val_accuracy: 0.5569 - val_loss: 1.1534
Epoch 10/50
448/448 ━━━━━━━━━━━━━━━━━━━━ 0s 185us/step - accuracy: 0.5938 - loss: 1.0554 - val_a

In [32]:
model_json = emotion_model.to_json()
with open('./model/emotion_model.json', 'w') as json_file:
    json_file.write(model_json)

emotion_model.save_weights('./model/emotion_model.weights.h5')

In [34]:
!pip install opencv-python

Defaulting to user installation because normal site-packages is not writeable
  Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl.metadata (20 kB)
Using cached opencv_python-4.10.0.84-cp37-abi3-win_amd64.whl (38.8 MB)


In [ ]:
#From below, codes are like this, we created a model above, stored it in a file, now we would like to use it somewhere else. So load the model from folder again and use it as done below.

In [35]:
import cv2
from keras.models import model_from_json

In [36]:
emotion_dict = {0:'Angry', 1:'Disgusted', 2:'Fearful', 3:'Happy', 4:'Neutral', 5:'Sad', 6:'Surprised'}

In [38]:
json_file = open('./model/emotion_model.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
emotion_model = model_from_json(loaded_model_json)
emotion_model.load_weights('./model/emotion_model.weights.h5')
print('Success, model loaded')

Success, model loaded


In [46]:
#start camera feed 
#cap = cv2.VideoCapture(0)

cap = cv2.VideoCapture(r"path")

while True:
    #haar cascade to draw box around face 
    ret, frame = cap.read()
    frame = cv2.resize(frame, (1280, 720))
    if not ret:
        break
    face_detector = cv2.CascadeClassifier('./haarcascades/haarcascade_frontalface_default.xml')
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    num_faces = face_detector.detectMultiScale(gray_frame, scaleFactor=1.3, minNeighbors=5)
    
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (0, 255, 0), 4)
        roi_gray_frame = gray_frame[y:y+h, x:x+w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        
        emotion_predicion = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_predicion))
        cv2.putText(frame, emotion_dict[maxindex], (x+5, y-20), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2, cv2.LINE_AA)

    cv2.imshow('Emotion Detection', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()


error: OpenCV(4.10.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4152: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'
